The purpose of this notebook is to build sklearn-like pipeline for transformation

In [5]:
import pandas as pd         
import os.path

N_JOBS = 7

In [6]:
def load_data(path="../data/csv/all.csv"):
    dataframe = pd.read_csv(path, index_col=0)
    return dataframe.loc[~dataframe["execTimeMs"].isnull()]

In [7]:
def prepare_dataframe(dataframe):
    output = dataframe.dropna(axis="columns")
    targets = output["execTimeMs"]
    dropped = output[["command", "execTimeMs", "jobId", "ctime_mean", "ctime_max", "ctime_sum", "read_sum","write_sum","readSyscalls_sum","writeSyscalls_sum","readReal_sum","writeReal_sum","writeCancelled_sum","rxBytes_sum","rxPackets_sum","rxErrors_sum","rxDrop_sum","rxFifo_sum","rxFrame_sum","rxCompressed_sum","rxMulticast_sum","txBytes_sum","txPackets_sum","txErrors_sum","txDrop_sum","txFifo_sum","txColls_sum","txCarrier_sum","txCompressed_sum","cpu_mean","cpu_max","memory_mean","memory_max"]]
    features = output.drop(dropped.columns, axis=1)
    return features, targets, dropped

In [8]:
features, targets, dropped = prepare_dataframe(load_data())

In [9]:
features.dtypes

workflowName          object
size                 float64
executable            object
args                  object
inputs                object
outputs               object
name                  object
cpu.manufacturer      object
cpu.brand             object
cpu.speed            float64
cpu.cores              int64
cpu.physicalCores      int64
cpu.processors         int64
mem.total              int64
mem.free               int64
mem.used               int64
mem.active             int64
mem.available          int64
mem.buffers            int64
mem.cached             int64
mem.slab               int64
mem.buffcache          int64
mem.swaptotal          int64
mem.swapused           int64
mem.swapfree           int64
dtype: object

# Preprocessing flow

In [10]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector

In [11]:
def vectorize_list(series):
    def vectorize(list_string):
        return len(eval(list_string))
    return np.vectorize(vectorize)(series)

def ListTransformer():
    return FunctionTransformer(func=vectorize_list)

In [12]:
list_transformer = Pipeline(steps=[("list", ListTransformer()), ("scaler", StandardScaler())])
list_features = list(['args', 'inputs', 'outputs'])

numerical_transformer = StandardScaler()
numerical_features = list(features.select_dtypes(include="number").columns)

categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")
categorical_features = list(set(features.select_dtypes(include="object").columns) ^ set(list_features))

def make_classifying_preprocessor(additional_features=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    external_features = categorical_features + additional_features
    return ColumnTransformer(
            transformers=[('lists', list_transformer, list_features), 
                          ('num', numerical_transformer, numerical_features),
                          ('cat', categorical_transformer, external_features)])

def make_regression_preprocessor(additional_features=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    external_features = numerical_features + additional_features
    return ColumnTransformer(
        transformers=[
            ('lists', list_transformer, list_features),            
            ('num', numerical_transformer, external_features),  
            ('cat', categorical_transformer, categorical_features)
        ])

preprocessor = make_classifying_preprocessor(additional_features=[])

In [13]:
from scipy.stats import percentileofscore
import math

def calculate_quantile_rank(labels, label):
    return percentileofscore(labels, label) / 100

def calculate_utilization_class(labels, label):
    def label_for_rank(rank):
        if rank > 0.75:
            return 'very high'
        elif rank > 0.5:
            return 'high'
        elif rank > 0.25:
            return 'medium'
        else:
            return 'low'
    return label_for_rank(calculate_quantile_rank(labels, label))

def calculate_utilization_bucket(labels, label, num_buckets):
    bucket_size = 1.0 / num_buckets
    def bucket_for_rank(rank):
        return str(math.floor(rank / bucket_size))
    return bucket_for_rank(calculate_quantile_rank(labels, label))

# Pipeline composition (with PCA)

In [14]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import Lasso, SGDRegressor, ElasticNet, LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.dummy import DummyRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, HalvingGridSearchCV

In [15]:
base_steps = [('pca', PCA(random_state=42))]
dummy_pipeline = Pipeline(steps=base_steps +[('dummy', DummyRegressor())])
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=0)

In [16]:
pca_param_grid = {
    'pca__n_components': np.arange(1, 50, 3),    
}
knn_param_grid = {
    'knn__n_neighbors': np.arange(1, 30, 3),
}
regressor = ('knn', KNeighborsRegressor())
full_pipeline = Pipeline(steps= base_steps + [regressor])
grid_search = HalvingGridSearchCV(full_pipeline, {**knn_param_grid, **pca_param_grid}, cv=2, verbose=2, scoring="r2", n_jobs=-1)

In [17]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
from loky import get_reusable_executor

def rate_regressor(X_train, y_train, X_test, y_test, regressor, regressor_params, verbose=10, aggressive_elimination=True, steps=base_steps):
    print(f"Rating {regressor}")
    full_pipeline = Pipeline(steps= base_steps + [regressor])
    vector_length = X_train.shape[1]
    pca_param_grid = {'pca__n_components': np.arange(1, vector_length, 1),}
    grid_search = HalvingGridSearchCV(full_pipeline, {**pca_param_grid, **regressor_params}, cv=2, verbose=verbose, scoring="r2", n_jobs=N_JOBS)
    print("Evaluating grid search")
    grid_search.fit(X_train, y_train)
    
    # scores
    print("Predicting on test set")
    prediction = grid_search.best_estimator_.predict(X_test)
    
    print("Calculating scores")
    executor = get_reusable_executor(max_workers=3, timeout=5)
    
    scores = [lambda true, pred: r2_score(true, pred), lambda true, pred: mean_absolute_error(true, pred), lambda true, pred: mean_absolute_percentage_error(true, pred)]
    results = executor.map(lambda fun: fun(y_test, prediction), scores)
    print("Calculated scores on test set")
    r2, mae, mape = list(results)
    adjusted_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    return {"r2": r2, "adjusted_r2": adjusted_r2, "mae": mae, "mape": mape,"best_score": grid_search.best_score_, "params": grid_search.best_params_}

In [18]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score

def rate_classifier(X_train, y_train, X_test, y_test, classifier, classifier_params, verbose=10, aggressive_elimination=True, steps=base_steps):
    print(f"Rating {classifier}")
    full_pipeline = Pipeline(steps= base_steps + [classifier])
    vector_length = X_train.shape[1]
    pca_param_grid = {'pca__n_components': np.arange(1, vector_length, 1),}
    grid_search = HalvingGridSearchCV(full_pipeline, {**pca_param_grid, **classifier_params}, cv=2, verbose=verbose, scoring="accuracy", n_jobs=N_JOBS)
    print("Evaluating grid search")
    grid_search.fit(X_train, y_train)
    
    # scores
    print("Predicting on test set")
    prediction = grid_search.best_estimator_.predict(X_test)
    
    print("Calculating scores")
    executor = get_reusable_executor(max_workers=5, timeout=5)
    
    scores = [
        lambda true, pred: f1_score(true, pred, average="micro"),
        lambda true, pred: f1_score(true, pred, average="macro"),
        lambda true, pred: accuracy_score(true, pred), 
        lambda true, pred: balanced_accuracy_score(true, pred)
    ]
    results = executor.map(lambda fun: fun(y_test, prediction), scores)
    print("Calculated scores on test set")
    micro, macro, accuracy, balanced_accuracy = list(results)
    return {"f1_micro": micro, "f1_macro": macro, "accuracy": accuracy, "balanced_accuracy": balanced_accuracy, "params": grid_search.best_params_}

# Here go regressor params

In [19]:
knn = ("knn", KNeighborsRegressor())
knn_params = {'knn__n_neighbors': np.arange(1, 30, 1)}

dtr = ("dtr", DecisionTreeRegressor(random_state=5))
dtr_params = {"dtr__criterion": ["mse", "friedman_mse", "mae", "poisson"]}

lasso = ("lasso", Lasso(random_state=5))
lasso_params = {"lasso__alpha": np.arange(0.01, 1, 0.05)}

en = ("elasticnet", ElasticNet(random_state=5))
en_params = {"elasticnet__alpha": np.arange(0.01, 1, 0.05), "elasticnet__l1_ratio": np.arange(0, 1, 0.1)}

svr = ("svr", SGDRegressor())
svr_params = {"svr__loss": ["squared_loss", "huber", "epsilon_insensitive"], "svr__penalty": ['l2', 'l1', 'elasticnet'],
             "svr__alpha": np.arange(0.0001, 0.2, 0.01), "svr__max_iter": [10000]}

rf = ("rf", RandomForestRegressor())
rf_params = {"rf__n_estimators": np.arange(5, 100, 5), "rf__criterion": ["mae", "mse"], "rf__max_features": ["auto", "sqrt", "log2"]}

# Here go classifier params

In [20]:
knn_classifier = ("knn", KNeighborsClassifier())
knn_clf_params = {'knn__n_neighbors': np.arange(1, 30, 1)}

dtr_classifier = ("dtr", DecisionTreeClassifier(random_state=5))
dtr_clf_params = {"dtr__criterion": ["gini", "entropy"]}

mlp_classifier = ("mlp", MLPClassifier())
mlp_clf_params = {"mlp__hidden_layer_sizes": np.arange(1,200, 10),          
                  "mlp__activation": ["logistic", "tanh", "relu"],         
                  "mlp__activation": ["logistic"],         
#                   "mlp__alpha": np.arange(0.01, 0.1, 0.01)
                 }

svc = ("svc", SVC(random_state=5))
svc_clf_params = {
    "svc__C": np.arange(0.1, 1, 0.1), 
    "svc__kernel": ["linear", "poly", "rbf", "sigmoid"],
    "svc__degree": np.arange(3, 10, 1)
}

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
def make_datasets(dataframe):
    jobs_below_1200ms = dataframe.loc[dataframe["execTimeMs"] < 1200]
    jobs_between_2000ms_25000ms = dataframe.loc[dataframe["execTimeMs"].between(2000, 25000)]
    jobs_count = dataframe["name"].value_counts()
    jobs_most_occuring = dataframe.loc[dataframe["name"].isin(jobs_count[jobs_count > 3000].index.values)]
    jobs_mDiffFit = dataframe.loc[dataframe["name"] == "mDiffFit"]
    jobs_haplotype = dataframe.loc[dataframe["name"] == "haplotype_caller"]
    jobs_mShrink = dataframe.loc[dataframe["name"] == "mShrink"]
    return jobs_below_1200ms, jobs_between_2000ms_25000ms, jobs_most_occuring

In [23]:
datasets = make_datasets(load_data())

In [24]:
dfs_for_jobs = [pd.DataFrame(y) for x, y in load_data().groupby('name', as_index=False)]

In [25]:
def rate_data(features, targets, regressors, verbose=10, pipeline_steps=base_steps):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=0)
    df = pd.DataFrame(columns=["name", "pca", "adjusted_r2","r2", "mae", "mape", "best_score", "params"])
    for (regressor, params) in regressors:
        result = rate_regressor(X_train, y_train, X_test, y_test, regressor, params, verbose, pipeline_steps)
        df = df.append({"name": regressor[0], **result, "pca": result["params"]["pca__n_components"]}, ignore_index=True)
    return df

In [26]:
def rate_classifiers_for_data(features, targets, classifiers, verbose=10, pipeline_steps=base_steps):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=0)
    df = pd.DataFrame(columns=["name", "pca", "accuracy","balanced_accuracy", "f1_micro", "f1_macro", "params"])
    for (classifier, params) in classifiers:
        result = rate_classifier(X_train, y_train, X_test, y_test, classifier, params, verbose, pipeline_steps)
        df = df.append({"name": classifier[0], **result, "pca": result["params"]["pca__n_components"]}, ignore_index=True)
    return df

In [27]:
def rate_dataset(dataframe, regressors, verbose=2):
    print(f"Rating dataset of len {len(dataframe)}")
    features, targets, _ = prepare_dataframe(dataframe[:10000])
    features = preprocessor.fit_transform(features)
    rate_data(features, targets, regressors, verbose)

In [28]:
basic_regressors = [
    (knn, knn_params),
#     (dtr, dtr_params),
    (lasso, lasso_params),
    (en, en_params),
    (svr, svr_params),
]

basic_classifiers = [
    (knn_classifier, knn_clf_params),
#     (dtr_classifier, dtr_clf_params),
    (mlp_classifier, mlp_clf_params),
    (svc, svc_clf_params)
]

In [29]:
def simple_experiment():
    print("Rating jobs datasets")
    for dataset in dfs_for_jobs:
        print(dataset.iloc[0]["name"])
        rate_dataset(dataset, basic_regressors)

    print("Rating common datasets")
    for dataset in datasets:
        rate_dataset(dataset, basic_regressors)

In [30]:
# rate_dataset(dfs_for_jobs[8], verbose=0)
# rate_dataset(dfs_for_jobs[1], basic_regressors, verbose=0)

***

## Eksperyment 2

### Cel

Dowiedzieć się jakich regresorów i klasyfikatorów używać do przewidywania poziomów zasobów

### Dane
krótsze niż 1200ms, 2k-25k ms, częstsze niż 3000, wszystkie

In [44]:
exp2_datasets = exp1_datasets

def get_pipeline_data_for_resource_buckets(data, resource, num_buckets=4):
    """
    Labels are resource utilization bucket indices.
    """
    features, labels, dropped = prepare_dataframe(data)
    labels = dropped[resource].map(lambda value: calculate_utilization_bucket(dropped[resource], value, num_buckets)).to_frame()
    features = make_classifying_preprocessor(additional_features=[]).fit_transform(features)
    return features, labels

def get_pipeline_data_for_continuous_score(data, resource):
    """
    Labels are quantile ranks of resource utilization
    """
    features, labels, dropped = prepare_dataframe(data)
    labels = dropped[resource].map(lambda value: calculate_quantile_rank(dropped[resource], value)).to_frame()
    features = make_classifying_preprocessor(additional_features=[]).fit_transform(features)
    return features, labels

### Przebieg

Znajdujemy najlepszy regresor dla każdego typu zasobów 

Znajdujemy najlepszy klasyfikator dla każdego typu zasobów przy 8 kubełkach

In [45]:
def run_experiment2a():
    exp2_resources = ["read_sum", "write_sum", "cpu_mean", "cpu_max", "memory_mean", "memory_max"]
    full_df = pd.DataFrame(columns=["dataset", "resource", "pipeline", "name", "pca", "adjusted_r2","r2", "mae", "mape", "best_score", "params"])
    
    for name, dataset in exp2_datasets.items():
        def run_regressor_pipeline(pipeline_name, features, targets):
            pipeline_df = rate_data(features, targets, basic_regressors, verbose=0)
            pipeline_df["pipeline"] = pipeline_name
            return pipeline_df
        
        print(f"\t\tEvaluating dataset {name} of length {len(dataset)}")
        for resource in exp2_resources:
            print(f"\tEvaluating resource {resource}")

            dataset_df = pd.concat([
                run_regressor_pipeline(f"continuous", *get_pipeline_data_for_continuous_score(dataset, resource)),
            ])
            dataset_df["dataset"] = name
            dataset_df["resource"] = resource
            full_df = full_df.append(dataset_df)
    return full_df

def run_experiment2b():
    exp2_resources = ["read_sum", "write_sum", "cpu_mean", "cpu_max", "memory_mean", "memory_max"]
    full_df = pd.DataFrame(columns=["dataset", "resource", "pipeline", "name", "pca",  "accuracy","balanced_accuracy", "f1_micro", "f1_macro", "params"])
    
    for name, dataset in exp2_datasets.items():
        def run_classifier_pipeline(pipeline_name, features, targets):
            pipeline_df = rate_classifiers_for_data(features, targets, basic_classifiers, verbose=0)
            pipeline_df["pipeline"] = pipeline_name
            return pipeline_df
        
        print(f"\t\tEvaluating dataset {name} of length {len(dataset)}")
        for resource in exp2_resources:
            print(f"\tEvaluating resource {resource}")

            dataset_df = pd.concat([
                run_classifier_pipeline(f"classifier_10", *get_pipeline_data_for_resource_buckets(dataset, resource, 10)),
            ])
            dataset_df["dataset"] = name
            dataset_df["resource"] = resource
            full_df = full_df.append(dataset_df)
    return full_df

In [46]:
if not os.path.isfile("data/exp2a.csv"):
    print("Running experiment 2a")
    exp2a_df = run_experiment2a()
    exp2a_df.to_csv("data/exp2a.csv")
else:
    exp2a_df = pd.read_csv("data/exp2a.csv").round(2).reset_index().drop(columns=['Unnamed: 0', 'index'])

if not os.path.isfile("data/exp2b.csv"):
    print("Running experiment 2b")
    exp2b_df = run_experiment2b()
    exp2b_df.to_csv("data/exp2b.csv")
else:
    exp2b_df = pd.read_csv("data/exp2b.csv").round(2).reset_index().drop(columns=['Unnamed: 0', 'index'])

Uwaga - osobny ran dla decision tree regressorów/klasyfikatorów


### Output eksperymentu
Regresory/klasyfikatory i ich parametry

In [47]:
exp2a_df.loc[exp2a_df.resource == 'memory_mean'].sort_values("r2", ascending=False)

,dataset,resource,pipeline,name,pca,adjusted_r2,r2,mae,mape,best_score,params
40,Between2KmsAnd25Kms,memory_mean,continuous,knn,60,0.81,0.82,0.09,0.86,0.81,"{'knn__n_neighbors': 4, 'pca__n_components': 60}"
42,Between2KmsAnd25Kms,memory_mean,continuous,elasticnet,57,0.80,0.81,0.10,0.95,0.80,"{'elasticnet__alpha': 0.01, 'elasticnet__l1_ra..."
43,Between2KmsAnd25Kms,memory_mean,continuous,svr,33,0.81,0.81,0.10,0.96,0.80,"{'pca__n_components': 33, 'svr__alpha': 0.0101..."
90,All,memory_mean,continuous,elasticnet,65,0.74,0.74,0.11,2.11,0.73,"{'elasticnet__alpha': 0.01, 'elasticnet__l1_ra..."
64,ExecutedMoreThan3Ktimes,memory_mean,continuous,knn,13,0.73,0.73,0.11,2.09,0.72,"{'knn__n_neighbors': 13, 'pca__n_components': 13}"
88,All,memory_mean,continuous,knn,26,0.73,0.73,0.11,1.92,0.73,"{'knn__n_neighbors': 6, 'pca__n_components': 26}"
91,All,memory_mean,continuous,svr,36,0.73,0.73,0.11,2.02,0.73,"{'pca__n_components': 36, 'svr__alpha': 0.0001..."
41,Between2KmsAnd25Kms,memory_mean,continuous,lasso,42,0.70,0.71,0.13,1.29,0.71,"{'lasso__alpha': 0.01, 'pca__n_components': 42}"
66,ExecutedMoreThan3Ktimes,memory_mean,continuous,elasticnet,40,0.71,0.71,0.12,2.33,0.71,"{'elasticnet__alpha': 0.01, 'elasticnet__l1_ra..."
67,ExecutedMoreThan3Ktimes,memory_mean,continuous,svr,19,0.70,0.70,0.12,2.29,0.70,"{'pca__n_components': 19, 'svr__alpha': 0.1201..."


In [48]:
exp2b_df.loc[exp2b_df.resource == 'memory_mean'].sort_values("accuracy", ascending=False)

,dataset,resource,pipeline,name,pca,accuracy,balanced_accuracy,f1_micro,f1_macro,params
31,Between2KmsAnd25Kms,memory_mean,classifier_10,mlp,49,0.51,0.50,0.51,0.49,"{'mlp__activation': 'logistic', 'mlp__hidden_l..."
30,Between2KmsAnd25Kms,memory_mean,classifier_10,knn,52,0.49,0.49,0.49,0.48,"{'knn__n_neighbors': 12, 'pca__n_components': 52}"
32,Between2KmsAnd25Kms,memory_mean,classifier_10,svc,37,0.49,0.49,0.49,0.49,"{'pca__n_components': 37, 'svc__C': 0.5, 'svc_..."
67,All,memory_mean,classifier_10,mlp,59,0.48,0.44,0.48,0.43,"{'mlp__activation': 'logistic', 'mlp__hidden_l..."
49,ExecutedMoreThan3Ktimes,memory_mean,classifier_10,mlp,31,0.47,0.47,0.47,0.45,"{'mlp__activation': 'logistic', 'mlp__hidden_l..."
66,All,memory_mean,classifier_10,knn,37,0.47,0.43,0.47,0.42,"{'knn__n_neighbors': 8, 'pca__n_components': 37}"
68,All,memory_mean,classifier_10,svc,43,0.47,0.42,0.47,0.40,"{'pca__n_components': 43, 'svc__C': 0.4, 'svc_..."
48,ExecutedMoreThan3Ktimes,memory_mean,classifier_10,knn,39,0.46,0.46,0.46,0.45,"{'knn__n_neighbors': 14, 'pca__n_components': 39}"
50,ExecutedMoreThan3Ktimes,memory_mean,classifier_10,svc,30,0.44,0.44,0.44,0.40,"{'pca__n_components': 30, 'svc__C': 0.30000000..."
13,ShorterThan1.2Kms,memory_mean,classifier_10,mlp,36,0.35,0.32,0.35,0.30,"{'mlp__activation': 'logistic', 'mlp__hidden_l..."
